In [1]:
import pandas as pd
import joblib
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import os

In [2]:
# 1. Crear carpeta de resultados si no existe
os.makedirs("models_test/saved", exist_ok=True)

In [5]:
# 2. Cargar datos
df = pd.read_csv("../Data/Data_Seattle.csv")

In [6]:
df.describe

<bound method NDFrame.describe of                id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7 

In [7]:
df.dtypes

id                 int64
date                 str
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [8]:
# 3. Procesar columna 'date'
df["date"] = pd.to_datetime(df['date'])
df["date"] = (df["date"] - df["date"].min()) / pd.Timedelta(days=1)

In [0]:
# # 3. Procesar columna 'date'
# df["date"] = pd.to_datetime(df["date"], format="%Y%m%dT%H%M%S", errors="coerce")
# df["year"] = df["date"].dt.year
# df["month"] = df["date"].dt.month
# df["day"] = df["date"].dt.day
# df = df.drop("date", axis=1)

In [9]:
# 4. Separar variables 
X = df.drop("price", axis=1)
y = df["price"]

In [10]:
stdscal = StandardScaler()
X = stdscal.fit_transform(X)

In [11]:
# 5. Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
param_grid = {'C':[0.1, 0.5, 1, 5, 10], 'kernel':('linear', 'rbf', 'poly'), 'epsilon':[0.1, 0.2, 0.35, 0.5]}
svr = SVR()
grid = GridSearchCV(svr, param_grid, verbose=3, return_train_score=True, scoring='neg_mean_squared_error')

In [13]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END C=0.1, epsilon=0.1, kernel=linear;, score=(train=-130427897108.576, test=-159681069290.714) total time=   5.3s
[CV 2/5] END C=0.1, epsilon=0.1, kernel=linear;, score=(train=-135653880618.866, test=-139597056085.453) total time=   5.3s
[CV 3/5] END C=0.1, epsilon=0.1, kernel=linear;, score=(train=-136498108582.405, test=-137350031435.254) total time=   5.2s
[CV 4/5] END C=0.1, epsilon=0.1, kernel=linear;, score=(train=-138257396288.452, test=-127886147224.588) total time=   5.0s
[CV 5/5] END C=0.1, epsilon=0.1, kernel=linear;, score=(train=-141206027699.087, test=-117547143855.401) total time=   5.3s
[CV 1/5] END C=0.1, epsilon=0.1, kernel=rbf;, score=(train=-132439001598.707, test=-161871390100.989) total time=   9.6s
[CV 2/5] END C=0.1, epsilon=0.1, kernel=rbf;, score=(train=-137541185646.444, test=-141495437585.392) total time=   9.3s
[CV 3/5] END C=0.1, epsilon=0.1, kernel=rbf;, score=(train=-138163609306.899

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.1, 0.5, ...], 'epsilon': [0.1, 0.2, ...], 'kernel': ('linear', ...)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is al

In [14]:
mejores_parametros = grid.best_params_
mejor_modelo = grid.best_estimator_
print(mejores_parametros)

{'C': 10, 'epsilon': 0.5, 'kernel': 'linear'}


In [15]:
# Realizar predicciones en el conjunto de prueba
y_pred = mejor_modelo.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2_best = r2_score(y_test, y_pred)
print(f"R2 Score: {r2_best}")

Mean Squared Error: 77541787963.74423
R2 Score: 0.4870784871151519


In [16]:
# 10. Guardar modelo 
joblib.dump(mejor_modelo, "models_test/saved/svm_gridsearch.pkl")

# 11. Guardar parámetros y métricas 
results = {
    "model": "SVM_GridSearch",
    "params": grid.best_params_,
    "metrics": {
        "RMSE": mse,
        "R²": r2_best
    }
}

with open("models_test/saved/svm_gridsearch_results.json", "w") as f:
    json.dump(results, f, indent=4)